In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset


In [2]:
from senmodel.model.utils import *
from senmodel.metrics.nonlinearity_metrics import *
from senmodel.metrics.edge_finder import *
from senmodel.metrics.train_metrics import *
from senmodel.train.train import *

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
from sklearn.preprocessing import LabelEncoder

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
    'hours-per-week', 'native-country', 'income'
]
data = pd.read_csv(url, names=columns, na_values=" ?", skipinitialspace=True)
data = data.dropna()

X = data.drop('income', axis=1)
y = data['income']

for col in X.select_dtypes(include=['object']).columns:
    X[col] = LabelEncoder().fit_transform(X[col])

y = LabelEncoder().fit_transform(y)

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)


In [5]:
class TabularDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.long)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

In [6]:
hyperparams = {
    "num_epochs": 100,
    "batch_size": 1024,
    "metric": AbsGradientEdgeMetric(nn.CrossEntropyLoss()),
    "aggregation_mode": "mean",
    "choose_thresholds": {"fc0": 0.7}, # 0.0 -> all edges, 1.0 -> no edges 
    "choose_thresholds_del": {"fc0": 0.1}, # 1.0 -> all edges, 0.0 -> no edges
    "threshold": 0.005,
    "min_delta_epoch_replace": 8,
    "window_size": 5,
    "lr": 5e-4,
    "delete_after": 5,    
    "task_type": "classification",
    "fully_connected": False,
    "max_to_replace": 900 # None -> no limit
}

name = ", ".join(
    f"{key}: {value.__class__.__name__ if key == 'metric' else value}"
    for key, value in hyperparams.items()
)

name

"num_epochs: 100, batch_size: 1024, metric: AbsGradientEdgeMetric, aggregation_mode: mean, choose_thresholds: {'fc0': 0.7}, choose_thresholds_del: {'fc0': 0.1}, threshold: 0.005, min_delta_epoch_replace: 8, window_size: 5, lr: 0.0005, delete_after: 5, task_type: classification, fully_connected: False, max_to_replace: 900"

In [7]:
train_dataset = TabularDataset(X_train, y_train)
val_dataset = TabularDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=hyperparams['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=hyperparams['batch_size'], shuffle=False)

In [8]:
class EnhancedFCN(nn.Module):
    def __init__(self, input_size=14, hidden_size=32):
        super(EnhancedFCN, self).__init__()
        self.fc0 = nn.Linear(input_size, hidden_size)
        self.fc1 = nn.Linear(hidden_size, 2)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.fc1(self.act(self.fc0(x)))
        return x

In [9]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: fedornigretuk to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [10]:
dense_model = EnhancedFCN()
sparse_model = convert_dense_to_sparse_network(dense_model, layers=[dense_model.fc0, dense_model.fc1])
wandb.finish()
wandb.init(
    project="self-expanding-nets-titanic",
    name=f"titanic, {name}",
    tags=["complex model", "titanic", "multiclass", hyperparams["metric"].__class__.__name__],
)

In [11]:
criterion = nn.CrossEntropyLoss()
train_sparse_recursive(sparse_model, train_loader, train_loader, val_loader, criterion, hyperparams, device)

100%|██████████| 26/26 [00:00<00:00, 59.67it/s]


Epoch 1/100, Train Loss: 0.7325, Val Loss: 0.6865, Val Accuracy: 0.5758


100%|██████████| 26/26 [00:00<00:00, 54.59it/s]


Epoch 2/100, Train Loss: 0.6558, Val Loss: 0.6238, Val Accuracy: 0.7522


100%|██████████| 26/26 [00:00<00:00, 41.66it/s]


...

Epoch 98/100, Train Loss: 0.3219, Val Loss: 0.3365, Val Accuracy: 0.8422
Chosen edges: tensor([[ 27,   0,   4,  26,   0,   5,  21,  26,  25,  21],
        [ 11,  49,  49,  49,  86,  86,  86,  86, 108, 168]]) 10


100%|██████████| 26/26 [00:01<00:00, 22.30it/s]


Epoch 99/100, Train Loss: 0.3221, Val Loss: 0.3366, Val Accuracy: 0.8420


100%|██████████| 26/26 [00:01<00:00, 25.98it/s]


Epoch 100/100, Train Loss: 0.3219, Val Loss: 0.3357, Val Accuracy: 0.8419
